In [192]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm
import pickle
from socceraction.data.statsbomb import StatsBombLoader
from statsbombpy import sb

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)
from express.databases import SQLiteDatabase
from express.visualization import plot_action
from express.utils import add_names
from functools import partial

from express.datasets import PressingDataset
from tqdm import tqdm
import pandas as pd
from pathlib import Path

import shap
import matplotlib.pyplot as plt


base_path: c:\Users\mirun\24_winter\press\sr-press


In [193]:
model = "xgboost" #"soccermap"
trial = 13 # as you like

path = Path(base_path) / "stores" / "model" / model / f"{trial:03d}" / "component.pkl"
with path.open(mode="rb") as file:
    component = pickle.load(file)
component

In [194]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")
test_db = SQLiteDatabase(TEST_DB_PATH)
dataset_test = partial(PressingDataset, path=os.path.join(base_path, "stores", "filtered_datasets", "test"))


In [195]:
print(component.features.keys())
component.test(dataset_test)

dict_keys(['closest_11_players', 'relative_startlocation', 'defenders_in_3m_radius', 'relative_endlocation', 'get_column_sum_to_player'])


{'precision': 0.4098360655737705,
 'recall': 0.08741258741258741,
 'f1': 0.1440922190201729,
 'log_loss': 0.6164863893636872,
 'brier': 0.21067506735907512,
 'roc_auc': 0.5854128411892265}

In [196]:
preds = component.predict(dataset_test)
preds

game_id  action_id
3857254  4            0.116563
         14           0.006936
         17           0.154682
         22           0.300543
         34           0.133272
                        ...   
3869685  3013         0.072382
         3020         0.301446
         3031         0.269038
         3041         0.212329
         3045         0.173190
Length: 14504, dtype: float32

In [197]:
test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "test"),
    xfns=["closest_11_players", "relative_startlocation", "defenders_in_3m_radius", "relative_endlocation", "get_column_sum_to_player"],
    yfns=["possession_change_by_5_seconds"],
    load_cached=True,
    nb_prev_actions=3
)

In [198]:
new = test_dataset.labels.copy()
new[f"{component.label[0]}_prob"] = preds.values
new


possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  
game_id action_id                                       
3857254 4                                     0.116563  
        14                                    0.006936  
        17                                    0.154682  
        22                                    0.300543  
        34                                    0.133272  
...                                                ...  
3869685 3013                                  0.072382  
        3020                                  0.301446  
        3031                                  0.269038  
        3041                                  0.212329  
        3045                                  0.173190  

[14504 rows x 2 columns]

### 압박 선수 id 구하기

### player_name 매핑해서 append하기

In [199]:
new.head(10)

possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
        37                                  False   
        40                                  False   
        46                                  False   
        55                                  False   
        90                                  False   

                   possession_change_by_5_seconds_prob  
game_id action_id                                       
3857254 4                                     0.116563  
        14                                    0.006936  
        17                                    0.154682  
        22                                    0.300543  
        34                                    0.133272  
        37                                    0.267218  
        40                                    0.333209  
        46                                    0.334076  
        55                                    0.323662  
        90                                    0.204938

In [200]:
loaded_teams_df=pd.read_csv('WC_test_player_id.csv')
new['player_id']=loaded_teams_df['player_id'].values
new

possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  player_id  
game_id action_id                                                  
3857254 4                                     0.116563      23910  
        14                                    0.006936      35592  
        17                                    0.154682       9236  
        22                                    0.300543      44955  
        34                                    0.133272       6302  
...                                                ...        ...  
3869685 3013                                  0.072382      24778  
        3020                                  0.301446       8519  
        3031                                  0.269038      16308  
        3041                                  0.212329      16308  
        3045                                  0.173190      38718  

[14504 rows x 3 columns]

In [201]:
import json
with open('players_2_id.json', 'r', encoding='utf-8') as file:
    team_mapping = json.load(file)
new['player_name'] = new['player_id'].astype(str).map(team_mapping)
new['xP_difference'] =  new[f"{component.label[0]}"] - new[f"{component.label[0]}_prob"]
new



possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  player_id  \
game_id action_id                                                   
3857254 4                                     0.116563      23910   
        14                                    0.006936      35592   
        17                                    0.154682       9236   
        22                                    0.300543      44955   
        34                                    0.133272       6302   
...                                                ...        ...   
3869685 3013                                  0.072382      24778   
        3020                                  0.301446       8519   
        3031                                  0.269038      16308   
        3041                                  0.212329      16308   
        3045                                  0.173190      38718   

                                player_name  xP_difference  
game_id action_id                                           
3857254 4                    Youssef Msakni      -0.116563  
        14                 Anis Ben Slimane      -0.006936  
        17                   Mohamed Dräger      -0.154682  
        22             Aïssa Bilal Laïdouni      -0.300543  
        34                   Kasper Dolberg      -0.133272  
...                                     ...            ...  
3869685 3013              Eduardo Camavinga      -0.072382  
        3020       Dayotchanculle Upamecano      -0.301446  
        3031         Leandro Daniel Paredes      -0.269038  
        3041         Leandro Daniel Paredes      -0.212329  
        3045                 Enzo Fernandez      -0.173190  

[14504 rows x 5 columns]

In [204]:

# 1. 팀당 pressing point 합 구하기
team_sum = new.groupby('player_name')[f"{component.label[0]}_prob"].sum().reset_index()

# 2. 팀당 xP_difference 합 구하기
team_difference = new.groupby('player_name')['xP_difference'].sum().reset_index()

# 3. 팀당 서로 다른 game_id 개수 세기
team_game_count = new.reset_index().groupby('player_name')['game_id'].nunique()
team_game_count_df = team_game_count.reset_index()


# 4. 팀당 서로 다른 action_id 개수 세기
team_action_count = new.reset_index().groupby('player_name')['action_id'].nunique()
team_action_count= team_action_count.reset_index()

# 데이터프레임 병합

merged_df = pd.merge(team_sum, team_difference, on='player_name', how='outer')  # 첫 번째 병합
merged_df = pd.merge(merged_df, team_game_count, on='player_name', how='outer')  # 두 번째 병합
merged_df = pd.merge(merged_df, team_action_count, on='player_name', how='outer')  # 세 번째 병합
merged_df.rename(columns={'game_id': 'games_played'}, inplace=True)
merged_df.rename(columns={'action_id': 'actions'}, inplace=True)
merged_df.rename(columns={f"{component.label[0]}_prob": 'xP_total'}, inplace=True)

# 추가 컬럼 계산
merged_df['xP_total']=merged_df['xP_total'].round(2)
merged_df['xP_difference']=merged_df['xP_difference'].round(2)
merged_df['xP_per_game'] = (merged_df['xP_total'] / merged_df['games_played']).round(2)
merged_df['xP_per_action'] = (merged_df['xP_total'] / merged_df['actions']).round(2)
merged_df['xp_difference_per_game'] = (merged_df['xP_difference'] / merged_df['games_played']).round(2)
merged_df['xp_difference_per_action'] = (merged_df['xP_difference'] / merged_df['actions']).round(2)
column_order = [
    'player_name',             # 선수 이름
    'games_played',            # 총 게임 수
    'actions',                 # 총 pressing 수    
    'xP_total',                # xP 합
    'xP_difference',           # xP_difference
    'xP_per_game',             # 경기당 xP
    'xp_difference_per_game',  # 경기당 xP_difference
    'xP_per_action',           # action당 xP
    'xp_difference_per_action' # action당 xP_difference 
]

# 열 순서를 재배치
merged_df = merged_df[column_order]


#merged_df.to_csv('WC_test_player_xP.csv', index=False)

In [207]:
# 정렬
condition=merged_df['games_played']>=3
merged_df[condition].sort_values('xp_difference_per_action', ascending=False).head(10)

,player_name,games_played,actions,xP_total,xP_difference,xP_per_game,xp_difference_per_game,xP_per_action,xp_difference_per_action
166,Edinson Roberto Cavani Gómez,3,15,3.25,6.75,1.08,2.25,0.22,0.45
227,Harry Wilson,3,15,4.41,5.59,1.47,1.86,0.29,0.37
335,Kendall Jamaal Waston Manley,3,19,4.94,6.06,1.65,2.02,0.26,0.32
574,Thiago Emiliano da Silva,4,18,6.24,5.76,1.56,1.44,0.35,0.32
22,Alejandro Balde Martínez,4,16,5.12,4.88,1.28,1.22,0.32,0.31
201,Gareth Frank Bale,3,16,4.46,4.54,1.49,1.51,0.28,0.28
306,João Maria Lobo Alves Palhinha Gonçalves,3,12,2.78,3.22,0.93,1.07,0.23,0.27
88,Boualem Khoukhi,3,17,5.54,4.46,1.85,1.49,0.33,0.26
198,Gabriel Fernando de Jesus,3,13,2.64,3.36,0.88,1.12,0.20,0.26
376,Marcos Aoás Corrêa,3,15,4.09,3.91,1.36,1.30,0.27,0.26
